# Limpieza de Datos y Análisis Exploratorio de Datos

#### **Librerías**

Importamos las librerías necesarias.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### **Lectura del dataset**

Leemos los datos.

In [2]:
df = pd.read_csv('delitos_2021.csv', sep=';')

Mostramos las cinco primeras filas del dataframe.

In [3]:
df.head()

,id-mapa,anio,mes,dia,fecha,franja,tipo,subtipo,uso_armas,barrio,comuna,latitud,longitud,cantidad
0,HC-2021-1008202,2021,Enero,Miércoles,06/01/2021,18,Homicidio,Siniestro Vial,NaN,Constitución,1.0,"-34,62761","-58,379146",1.0
1,HC-2021-1009094,2021,Enero,Jueves,07/01/2021,10,Homicidio,Siniestro Vial,NaN,Mataderos,9.0,"-34,64972969","-58,51058958",1.0
2,HC-2021-1013236,2021,Enero,Domingo,10/01/2021,7,Homicidio,Siniestro Vial,NaN,Parque Chacabuco,7.0,"-34,62917196","-58,43058079",1.0
3,HC-2021-1014154,2021,Enero,Lunes,11/01/2021,9,Homicidio,Siniestro Vial,NaN,Nueva Pompeya,5.0,"-34,63867577","-58,4145554",1.0
4,HC-2021-1015476,2021,Enero,Lunes,11/01/2021,21,Homicidio,Siniestro Vial,NaN,Parque Avellaneda,9.0,"-34,64771436","-58,46806552",1.0


#### **Análisis del Dataset**

Consultamos el tamaño del dataset.

In [4]:
df.shape

(88567, 14)

Descripción del Dataset

In [5]:
df.describe()

,anio,comuna,cantidad
count,88567.0,87907.000000,7299.000000
mean,2021.0,7.538899,1.158515
std,0.0,4.554293,0.557664
min,2021.0,0.000000,1.000000
25%,2021.0,4.000000,1.000000
50%,2021.0,7.000000,1.000000
75%,2021.0,12.000000,1.000000
max,2021.0,15.000000,10.000000


#### **Tipos de Datos**

Tipos de datos de las columnas.

In [6]:
df.dtypes

id-mapa       object
anio           int64
mes           object
dia           object
fecha         object
franja        object
tipo          object
subtipo       object
uso_armas     object
barrio        object
comuna       float64
latitud       object
longitud      object
cantidad     float64
dtype: object

Vemos que la columna fecha tiene como tipo de dato object, cambiamos el tipo de dato a datatime.

In [7]:

df['fecha'] = df['fecha'].apply(lambda x: pd.to_datetime(x, errors='coerce', utc=True))

/var/folders/xj/j8r00dd95wn7lh34b83jtgg80000gp/T/ipykernel_93506/1961818311.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['fecha'] = df['fecha'].apply(lambda x: pd.to_datetime(x, errors='coerce', utc=True))


In [8]:
df.dtypes

id-mapa                   object
anio                       int64
mes                       object
dia                       object
fecha        datetime64[ns, UTC]
franja                    object
tipo                      object
subtipo                   object
uso_armas                 object
barrio                    object
comuna                   float64
latitud                   object
longitud                  object
cantidad                 float64
dtype: object

In [9]:
### Comuna. Cambiar o no a números? Latitud y Longitud lo mismo?

#### **Valores Nulos**

In [10]:
df.isnull().sum()

id-mapa          0
anio             0
mes              0
dia              0
fecha            0
franja           0
tipo             0
subtipo      72246
uso_armas    80095
barrio        1177
comuna         660
latitud          0
longitud         0
cantidad     81268
dtype: int64

Podemos ver que las columnas subtipo, uso_armas, barrio y comuna tienen valoes nulos.

Analizamos los valores nulos de la columna barrio y comuna.

In [15]:
df[df.barrio.isnull()].comuna.value_counts()

comuna
0.0    517
Name: count, dtype: int64

In [17]:
df[df.barrio.isnull()].comuna.isnull().sum()

660

Podemos observar que todas las filas vacías de la columna barrio tienen la columna comuna vacía o con 0.  Es decir en las 1177 filas de barrios nulas, no hay el dato correspondiente a comuna. 
Ahora analizamos las columnas latitud y longitud de las mismas filas.

In [18]:
df[df.barrio.isnull()].latitud.value_counts()

latitud
SD    1177
Name: count, dtype: int64

In [19]:
df[df.barrio.isnull()].longitud.value_counts()

longitud
SD    1177
Name: count, dtype: int64

Como vemos tampoco tienen los datos de longitud y latitud, por lo tanto deberíamos borrar estas filas.

In [20]:
df.dropna(subset=['barrio'], inplace = True)

In [21]:
df.barrio.isnull().sum()

0

In [23]:
df.isnull().sum()

id-mapa          0
anio             0
mes              0
dia              0
fecha            0
franja           0
tipo             0
subtipo      71631
uso_armas    78992
barrio           0
comuna           0
latitud          0
longitud         0
cantidad     80609
dtype: int64

In [24]:
df.subtipo.value_counts()

subtipo
Siniestro Vial     6683
Con uso de moto    4588
Hurto Automotor    3666
Robo Automotor      724
Doloso               84
Femicidio            14
Name: count, dtype: int64

In [25]:
df.uso_armas.value_counts()

uso_armas
Arma de fuego    5917
Arma cortante    2481
Name: count, dtype: int64

Observamos que aún tenemos valores nulos en las columnas subtipo y uso_armas, no tenemos manera de encontrar esta información en otro lugar. Las filas en las que tenemos información nos servirán para el análisis de datos.